In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [2]:

# ============================================================
# 0. Path 설정
# ============================================================
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

RAW_PATH      = DATA_DIR / "raw" / "train.csv"
CLEAN_PATH    = DATA_DIR / "interim" / "train_clean_v2.csv"
MONTHLY_PATH  = DATA_DIR / "processed" / "train_monthly_v2.csv"

print("📂 RAW_PATH:", RAW_PATH)

# ============================================================
# 1. 데이터 로드 & 기본 타입 정리
# ============================================================
df = pd.read_csv(RAW_PATH)

df["year"]     = df["year"].astype(int)
df["month"]    = df["month"].astype(int)
df["seq"]      = df["seq"].astype(int)
df["type"]     = df["type"].astype(str)
df["hs4"]      = df["hs4"].astype(str).str.zfill(4)
df["weight"]   = df["weight"].astype(float)
df["quantity"] = df["quantity"].astype(float)
df["value"]    = df["value"].astype(float)

print(f"✅ Loaded: {df.shape}")

# 결측 0 채우기 (필요시)
missing_before = df.isna().sum().sum()
df = df.fillna(0)
print(f"🧩 NA: {missing_before} → {df.isna().sum().sum()}")

# 완전 중복 제거
before = len(df)
df = df.drop_duplicates()
print(f"🔁 Drop duplicates: {before} → {len(df)}")

# ============================================================
# 2. HS 계층 파생 + 논리 불일치 플래그
# ============================================================
df["hs2"] = df["hs4"].str[:2]
df["hs3"] = df["hs4"].str[:3]

df["flag_v0_wpos"] = ((df["value"] == 0) & (df["weight"] > 0)).astype(int)
df["flag_w0_vpos"] = ((df["weight"] == 0) & (df["value"] > 0)).astype(int)

print("⚠️ value=0 & weight>0:", df["flag_v0_wpos"].sum())
print("⚠️ weight=0 & value>0:", df["flag_w0_vpos"].sum())

# ============================================================
# 3. 월별 집계 (item_id, type, hs4 기준)
# ============================================================
group_cols = ["item_id", "type", "hs4", "hs3", "hs2", "year", "month"]

monthly = (
    df.groupby(group_cols, as_index=False)
      .agg(
          total_value    = ("value", "sum"),
          total_weight   = ("weight", "sum"),
          total_quantity = ("quantity", "sum"),
          flag_v0_wpos   = ("flag_v0_wpos", "max"),
          flag_w0_vpos   = ("flag_w0_vpos", "max"),
      )
      .sort_values(group_cols)
)

print(f"📦 Monthly shape: {monthly.shape}")

# ============================================================
# 4. 로그/비율 피처
# ============================================================
monthly["log_value"]    = np.log1p(monthly["total_value"])
monthly["log_weight"]   = np.log1p(monthly["total_weight"])
monthly["log_quantity"] = np.log1p(monthly["total_quantity"])

monthly["wv_ratio"] = monthly["total_weight"] / np.maximum(monthly["total_value"], 1)

# ============================================================
# 5. KMeans 클러스터 (value–weight 패턴 4개)
# ============================================================
mask_pos = (monthly["total_value"] > 0) & (monthly["total_weight"] > 0)
X = monthly.loc[mask_pos, ["log_weight", "log_value"]].values

print(f"🤖 KMeans 학습용 샘플 수: {X.shape[0]}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42, n_init="auto")
monthly["cluster_wv"] = -1  # 기본값
monthly.loc[mask_pos, "cluster_wv"] = kmeans.fit_predict(X_scaled)

monthly["cluster_wv"] = monthly["cluster_wv"].astype(int)

# ============================================================
# 6. 저장
# ============================================================
CLEAN_PATH.parent.mkdir(parents=True, exist_ok=True)
MONTHLY_PATH.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(CLEAN_PATH, index=False)
monthly.to_csv(MONTHLY_PATH, index=False)

print(f"💾 train_clean_v2 saved to: {CLEAN_PATH}")
print(f"💾 train_monthly_v2 saved to: {MONTHLY_PATH}")


📂 RAW_PATH: /data/ephemeral/home/data/raw/train.csv
✅ Loaded: (10836, 9)
🧩 NA: 0 → 0
🔁 Drop duplicates: 10836 → 10836
⚠️ value=0 & weight>0: 1
⚠️ weight=0 & value>0: 300
📦 Monthly shape: (3776, 12)
🤖 KMeans 학습용 샘플 수: 3685
💾 train_clean_v2 saved to: /data/ephemeral/home/data/interim/train_clean_v2.csv
💾 train_monthly_v2 saved to: /data/ephemeral/home/data/processed/train_monthly_v2.csv
